In [1]:
#Importing required libraries.

import pandas as pd
import os
import string
from collections import Counter
import sklearn
import nltk
from natsort import natsorted
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import PySimpleGUI as sg

This code is used to open existing RAW article and filter it. We are removing stop words, and splitting text into list of words.

In [2]:
def articleClean(file):
    stop_words = set((stopwords.words('english')) + list(string.punctuation))

    symbols = ["\n", "’", "“", "”", "'", "`", "-", "[", "]", "»", ":"] 

    article = ""
    
    for i in symbols:
        file = file.replace(i, " ")

    def listToString(s): #function that converts arr of strings into one string (used later)
        str1 = " "
        return (str1.join(s))
    
    word_tokens= word_tokenize(file.lower()) #converts arcivle into array of words

    filtered_article = [] 

    for w in word_tokens: # check if word from article is the extra , if not assign the word in the clean words array
        if w not in stop_words and w.isalpha() == True:
            filtered_article.append(w)

    article = listToString(filtered_article)

    return article

Creating lists of articles and genres for training data.

In [4]:
all_articles = [] #training articles
genres = [] #genres

Store all technology articles in list and apply cleaning method.

In [5]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/technology/training/") #access technology articles directory
#tech_res_art = []  
for i in range(1,101):
    tech_res = open("training_tech" + str(i) + ".txt", "r").read() #Read articles
    tech_res = articleClean(tech_res) #Apply cleaning to all articles
    #tech_res_art.append(tech_res)
    all_articles.append(tech_res) #Add articles into the list
    genres.append("technology") #Add 'technology' to genres

Store all sport articles in list and apply cleaning method.

In [6]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/sport/training/")
sport_res_art = []

for i in range(1,101):
    sport_res = open("training_sport" + str(i) + ".txt", "r").read()
    sport_res = articleClean(sport_res)
    sport_res_art.append(sport_res)
    all_articles.append(sport_res)
    genres.append("sport")

Create DataFrame.

In [7]:
d = {'articles' : all_articles, 'genres' : genres}  
training_df = pd.DataFrame(data=d)  #Create training DataFrame
training_df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


Make everything lowercase.

In [8]:
training_df["articles"] = training_df["articles"].apply(lambda str : str.lower())
training_df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


In [9]:
training_df['label'] = training_df['genres'].apply(lambda x: 0 if x=="sport" else 1) #Create column 'label'. If genre is tech => label is 1, if sport => 0.
training_df  

,articles,genres,label
0,european commission clarified claims perceived...,technology,1
1,audio player european union investigates micro...,technology,1
2,mentioned previously european commission embar...,technology,1
3,european commission responded reports perceive...,technology,1
4,god war ragnarok released days ago obviously s...,technology,1
...,...,...,...
195,dallas luka doncic admitted fatigue following ...,sport,0
196,portland trail blazers finished long six game ...,sport,0
197,nothing alleviates hangover nasty road trip li...,sport,0
198,arizona running back michael wiley right fends...,sport,0


Create DataFrame for testing data.

In [10]:
test_articles = []
test_genres = []

In [11]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/technology/testing/")
for i in range(1, 101):
    test_tech_res = open("testing_tech" + str(i) + ".txt", "r").read()
    test_tech_res = articleClean(test_tech_res)
    test_articles.append(test_tech_res)
    test_genres.append("technology")

In [12]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/sport/testing/")
for i in range(1, 101):
    test_sport_res = open("testing_sport" + str(i) + ".txt", "r").read()
    test_sport_res = articleClean(test_sport_res)
    test_articles.append(test_sport_res)
    test_genres.append("sport")

In [13]:
d = {'articles' : test_articles, 'genres' : test_genres}
test_df = pd.DataFrame(data = d)
test_df

,articles,genres
0,might already seen pokémon scarlet violet come...,technology
1,battle larry normal type gym leader medali vic...,technology
2,son desperately wanted try game give bought di...,technology
3,something look forward interview pc world chie...,technology
4,powercolor teased next gen radeon rx xtx red d...,technology
...,...,...
195,eugene bo nix started week hobbled spectator s...,sport
196,estimated read time minutes eugene fourth pac ...,sport
197,red hot vegas golden knights edmonton saturday...,sport
198,vegas golden knights lost third time last four...,sport


In [16]:
test_df['label'] = test_df['genres'].apply(lambda x: 0 if x=="sport" else 1)
test_df

,articles,genres,label
0,might already seen pokémon scarlet violet come...,technology,1
1,battle larry normal type gym leader medali vic...,technology,1
2,son desperately wanted try game give bought di...,technology,1
3,something look forward interview pc world chie...,technology,1
4,powercolor teased next gen radeon rx xtx red d...,technology,1
...,...,...,...
195,eugene bo nix started week hobbled spectator s...,sport,0
196,estimated read time minutes eugene fourth pac ...,sport,0
197,red hot vegas golden knights edmonton saturday...,sport,0
198,vegas golden knights lost third time last four...,sport,0


In [17]:
df = training_df 
df = training_df.append(test_df, ignore_index=True)#  
df

/tmp/ipykernel_24174/3196510612.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = training_df.append(test_df, ignore_index=True)#


,articles,genres,label
0,european commission clarified claims perceived...,technology,1
1,audio player european union investigates micro...,technology,1
2,mentioned previously european commission embar...,technology,1
3,european commission responded reports perceive...,technology,1
4,god war ragnarok released days ago obviously s...,technology,1
...,...,...,...
395,eugene bo nix started week hobbled spectator s...,sport,0
396,estimated read time minutes eugene fourth pac ...,sport,0
397,red hot vegas golden knights edmonton saturday...,sport,0
398,vegas golden knights lost third time last four...,sport,0


Calculate the probability of article being about sport or technology.

In [19]:
sum_s = 0
sum_t = 0
for i in df['genres']:
    if i == "sport":
        sum_s +=1
    else:
        sum_t +=1

prob_s = sum_s/len(df)
prob_t = sum_t/len(df)
print(prob_s)
print(prob_t)

0.5
0.5


In [18]:
from sklearn.model_selection import train_test_split
taining_x, X_test, y_train, y_test = train_test_split(df['articles'], df['label'], random_state=1) #X - articles fortesting and training y- labels for testing and training
y_train

82     1
367    0
179    0
27     1
89     1
      ..
255    1
72     1
396    0
235    1
37     1
Name: label, Length: 300, dtype: int64

In [20]:
cv = CountVectorizer(strip_accents="ascii") # tokenizes the documents andconverts it to matrix
taining_x_cv = cv.fit_transform(taining_x) #Learn the vocabulary dictionary and return document-term matrix.
X_test_cv = cv.transform(X_test) #Transform documents to document-term matrix. according to the previous fit_transform() standard
print(X_test_cv)

  (0, 16)	2
  (0, 23)	1
  (0, 163)	1
  (0, 221)	1
  (0, 260)	1
  (0, 262)	1
  (0, 411)	2
  (0, 543)	1
  (0, 591)	2
  (0, 644)	2
  (0, 706)	1
  (0, 709)	2
  (0, 723)	1
  (0, 858)	1
  (0, 870)	1
  (0, 1086)	1
  (0, 1240)	1
  (0, 1434)	2
  (0, 1472)	2
  (0, 1480)	2
  (0, 1498)	1
  (0, 1519)	1
  (0, 1523)	1
  (0, 1603)	1
  (0, 1630)	2
  :	:
  (99, 10895)	1
  (99, 10980)	3
  (99, 10982)	1
  (99, 11022)	1
  (99, 11098)	1
  (99, 11111)	2
  (99, 11211)	1
  (99, 11220)	1
  (99, 11339)	12
  (99, 11341)	2
  (99, 11342)	2
  (99, 11397)	2
  (99, 11400)	1
  (99, 11429)	1
  (99, 11451)	1
  (99, 11464)	4
  (99, 11480)	1
  (99, 11524)	8
  (99, 11544)	1
  (99, 11572)	2
  (99, 11610)	2
  (99, 11611)	1
  (99, 11633)	1
  (99, 11698)	5
  (99, 11703)	5


In [21]:
word_freq_df = pd.DataFrame(taining_x_cv.toarray(), columns=cv.get_feature_names())
top_words_df = pd.DataFrame(word_freq_df.sum()).sort_values(0, ascending=False)
# print(word_freq_df["apple"][130:200])
# i = 0
# while i < 200:
#     if word_freq_df["aac"][i] !=0:
#         print(word_freq_df["aac"][i])
print(word_freq_df)
print(top_words_df)

     ETCHED  FOREVER  HISTORY  IN  aaa  aac  aaron  aaronson  abandon  \
0         0        0        0   0    0    0      0         0        0   
1         0        0        0   0    0    0      0         0        0   
2         0        0        0   0    0    0      0         0        0   
3         0        0        0   0    0    0      0         0        0   
4         0        0        0   0    0    0      0         0        0   
..      ...      ...      ...  ..  ...  ...    ...       ...      ...   
295       0        0        0   0    0    0      0         0        0   
296       0        0        0   0    0    0      0         0        0   
297       0        0        0   0    0    0      0         0        0   
298       0        0        0   0    0    0      0         0        0   
299       0        0        0   0    0    0      0         0        0   

     abandoned  ...  zombie  zone  zones  zonli  zoom  zooming  zr  \
0            0  ...       0     0      0      0     0

/home/farkhad/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB() # declare naive beyes model
naive_bayes.fit(taining_x_cv, y_train) #trian naive beyes model
predictions = naive_bayes.predict(X_test_cv) #predict
ara = naive_bayes.predict(X_test_cv[0])
print(ara)
predictions

[0]


array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [23]:
#display accuracy Percision and Recall
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy ='Accuracy score: ' + str(accuracy_score(y_test, predictions))
recallScore = 'Recall score: '+ str(recall_score(y_test, predictions))
percisionScore = 'Precision score: ' + str(precision_score(y_test, predictions))

print(accuracy)
print(recallScore)
print(percisionScore)

Accuracy score: 1.0
Recall score: 1.0
Precision score: 1.0


In [24]:
preds = [] #array later column which will contain predictions
for i in range(len(X_test)): #loop length = total amound of documents
    if predictions[i] == 1: #if predicion of that document is 1 which means it's a technology append preds with tech else append woth sports
        preds.append('technology')
    else:
        preds.append('sports')
predictedData = pd.DataFrame({'actual label': list(y_test), 'prediction': preds, 'article':list(X_test)}) #init new dataframe with three columns actual label and so on
predictedData.replace(to_replace=0, value='sports', inplace=True) #replace 0s in table with sports
predictedData.replace(to_replace=1, value='technology', inplace=True) #replace 1s in table with tech

predictedData

,actual label,prediction,article
0,sports,sports,vegas golden knights lost third time last four...
1,sports,sports,ann arbor ronnie bell caught touchdown extend ...
2,sports,sports,columbia ap spencer rattler threw yards six to...
3,sports,sports,controversial call late illinois game michigan...
4,sports,sports,brandin cooks still texans wide receiver repor...
...,...,...,...
95,sports,sports,week nfl season features massive slate games m...
96,technology,technology,black friday gigabyte rtx gaming laptop opens ...
97,technology,technology,whole reason people agenst whole cloud version...
98,technology,technology,buy links insider may earn affiliate commissio...


In [38]:
#function that predicts the genre of a singular article

#example sports article
textSports = 'Just when it seemed like the top 10 was going to escape unblemished after TCU and Michigan both hit late field goals to stay undefeated, Week 12 delivered the drama.'

#example tech article
textTech = 'It has been a while since Instagram introduced a website to let users access the social network from anywhere through a web browser. Over time, Instagram Web has been updated with many new features. This week, the Instagram website has been completely redesigned to take advantage of larger screens.'

def predictArticle(article):

    pred = 0 #prediction value that function returns

    clean_article = articleClean(article) #clean the article from extra words and chars
    clean_article = [clean_article] #redeclare article as an iterable object (list)

    cv_article = cv.transform(clean_article) #convert clean article into an array of features

    # print(cv_article)

    temp_NB = naive_bayes.predict(cv_article) #predict the genre of the article

    pred = temp_NB[0] #set value of pred to prediction value from Naive beyes
    # print("prediction: ", pred)

    #returrns 1 if Tech and 0 if Sports

    return pred

#predictArticle(textSports)
predictArticle(textTech)

1

creating Main method for UI.

In [37]:
layout = [[sg.Text('Enter article number: ')],      
                 [sg.InputText()],      
                 [sg.Submit(), sg.Cancel()]]      


window = sg.Window('NaiveBaise predictor.', layout)    

event, values = window.read()    
window.close()

text_input = values[0]
final_output = ""
answer = predictArticle(text_input)
if answer ==1:
    final_output = "It is a tech article"
else: 
    final_output = "It is a sport article"
sg.popup(final_output)

'OK'